In [8]:
import sys
import dataclasses
import requests
import logging
import base64
import pandas as pd
import openai
import os
from dotenv import load_dotenv
from tqdm import tqdm

sys.path.append('..')
import nature_go_client
from summary_generation import generate_summaries

logger = logging.getLogger(__name__)

load_dotenv()

# Your id here
NG_USERNAME = os.getenv("NG_USERNAME")
NG_PASSWORD = os.getenv("NG_PASSWORD")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
client = nature_go_client.NatureGoClient(username=NG_USERNAME, password=NG_PASSWORD)
client.login()

In [13]:
species_list = pd.DataFrame(client.get_labeled_species(limit=10, descriptions=False))
species_list = species_list[species_list['num_observations_total'] > 0]
species_list

,id,scientificNameWithoutAuthor,commonNames,genus,family,gbif_id,powo_id,wikipedia_word_count,number_of_occurrences,occurences_cdf,...,descriptions,illustration,illustration_transparent,illustration_reference,illustration_reference_transparent,reference_image_url,illustration_url,display_name,num_observations_total,num_questions_total
0,281,Astrantia major,"[Astrantia, Greater masterwort, Great Masterwort]",Astrantia,Apiaceae,3034915.0,838340-1,603.0,1457.0,0.475231,...,[],http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,http://nature-go.edouardleurent.com/media/spec...,Astrantia,1,0
1,404,Rosa spinosissima,"[Burnet rose, Scotch rose, Scotchbriar]",Rosa,Rosaceae,3002333.0,324822-2,274.0,1067.0,0.380851,...,[],http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,http://nature-go.edouardleurent.com/media/spec...,Burnet rose,1,0
2,893,Crambe maritima,"[Sea kale, Sea Cabbage, Kale]",Crambe,Brassicaceae,5373533.0,281657-1,1117.0,372.0,0.190132,...,[],http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,http://nature-go.edouardleurent.com/media/spec...,Sea kale,1,0
3,1131,Hedera colchica,"[Persian Ivy, Colchis ivy, Caucasian Ivy]",Hedera,Araliaceae,3036033.0,90679-1,419.0,255.0,0.145775,...,[],http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,http://nature-go.edouardleurent.com/media/spec...,Persian Ivy,1,0
4,2271,Ocimum basilicum,"[Sweet basil, Basil, Thai Basil]",Ocimum,Lamiaceae,2927096.0,452874-1,2043.0,63.0,0.055122,...,[],http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,http://nature-go.edouardleurent.com/media/spec...,Sweet basil,1,0
5,4127,Paeonia lactiflora,"[Common garden peony, Peony, Chinese peony]",Paeonia,Paeoniaceae,3083486.0,711802-1,490.0,14.0,0.019946,...,[],None,None,None,None,,http://nature-go.edouardleurent.com/static/img...,Common garden peony,1,0
6,36470,Stellaria holostea,"[Greater stichwort, Greater Stitchwort, Easter...",Stellaria,Caryophyllaceae,,,NaN,NaN,NaN,...,[],None,None,None,None,,http://nature-go.edouardleurent.com/static/img...,Greater stichwort,1,0


In [15]:
for _, species in tqdm(species_list.iterrows()):
    species_name, species_id = species['scientificNameWithoutAuthor'], species['id']
    summaries = generate_summaries(species_name)
    client.post_species_descriptions(species_id, summaries)
    print(species_name)

1it [00:19, 20.00s/it]

Astrantia major


2it [00:42, 21.64s/it]

Rosa spinosissima


3it [01:03, 21.12s/it]

...

Ocimum basilicum


6it [02:25, 24.54s/it]

Paeonia lactiflora


7it [02:40, 22.89s/it]

Stellaria holostea


## New generation 

In [10]:
import pandas as pd

df = pd.DataFrame(client.get_labeled_species(limit=1000, descriptions=False))

print(df['wikipedia_word_count'].sum(), "total words")
print(df['wikipedia_word_count'].sum() / 1000 * 0.002, "total wiki cost")

730210 total words
1.46042 total wiki cost
